In [1]:
#!pip install transformers

from transformers import BertForSequenceClassification

output_dir = 'E:\Grammar Classifier\saved_model-20220510T081104Z-001\saved_model\BERT'

print(output_dir)

E:\Grammar Classifier\saved_model-20220510T081104Z-001\saved_model\BERT


In [4]:
#!pip install torch

You should consider upgrading via the 'c:\users\manan\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
from transformers import BertTokenizer
import torch
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(output_dir)
model_loaded = BertForSequenceClassification.from_pretrained(output_dir)

Loading BERT tokenizer...


In [4]:
import pandas as pd

test=pd.read_csv('E:\Grammar Classifier\saved_model-20220510T081104Z-001\saved_model\BERT\\review_data.csv')

In [5]:
test.isna().sum()

text          1
star          0
app_id        0
reviewDate    0
dtype: int64

In [6]:
test.dropna(inplace=True)

In [8]:
sentences=test.text
sentences

0                                          Anathi Khanyile
1        Tony bahut funny hai Hill climbing racing my f...
2                                                  Teturwu
3        Hoooooooooooyaaaaaaaaa what a game hoooooooooo...
4                                        This game is nice
                               ...                        
29995                                   Plz my Vidos viral
29996                                                 Nice
29997                    Disturbing too much as YouTube ad
29998                                        Koub valo vai
29999                            I want to grow my account
Name: text, Length: 29999, dtype: object

In [9]:
predictions=[]

for n in range(0,30):
    input_ids=[]
    attention_masks=[]
    for sent in sentences[n*1000:(n+1)*1000]:
        encoded_dict = tokenizer.encode_plus(
                                  sent,                      # Sentence to encode.
                                  add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  max_length = 64,           # Pad & truncate all sentences.
                                  pad_to_max_length = True,
                                  return_attention_mask = True,   # Construct attn. masks.
                                  return_tensors = 'pt',     # Return pytorch tensors.
                            )

              # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

              # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    with torch.no_grad():
                # Forward pass, calculate logit predictions
            outputs = model_loaded(input_ids, token_type_ids=None, 
                              attention_mask=attention_masks)  
    logits = outputs[0]

    for logit in logits:
        index = logit.argmax()
        if index == 1:
            predictions.append("Gramatically correct")
        else:
            predictions.append("Gramatically in-correct")



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Manan\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
len(predictions)

29999

In [16]:
result=pd.concat([test,predictions],axis=1)

In [26]:
result.columns

Index(['text', 'star', 'app_id', 'reviewDate', 0], dtype='object')

In [56]:
result.reset_index(drop=True,inplace=True)

In [57]:
result['label']=y_pred

In [58]:
result

,text,star,app_id,reviewDate,label
0,Anathi Khanyile,5.0,com.fingersoft.hillclimb,18/03/21,Gramatically correct
1,Tony bahut funny hai Hill climbing racing my f...,5.0,com.fingersoft.hillclimb,18/03/21,Gramatically correct
2,Teturwu,1.0,com.fingersoft.hillclimb,18/03/21,Gramatically correct
3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...,5.0,com.fingersoft.hillclimb,18/03/21,Gramatically correct
4,This game is nice,5.0,com.fingersoft.hillclimb,18/03/21,Gramatically correct
...,...,...,...,...,...
29994,Plz my Vidos viral,5.0,video.like,21/03/21,Gramatically in-correct
29995,Nice,5.0,video.like,21/03/21,Gramatically correct
29996,Disturbing too much as YouTube ad,1.0,video.like,21/03/21,Gramatically in-correct
29997,Koub valo vai,5.0,video.like,21/03/21,Gramatically correct


In [63]:
result.to_csv('result.csv',index=False)